In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

chrome_options = Options()
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--headless")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
PAGES = range(1,4)
data=[]
for PAGE in PAGES:
    URL = f"https://www.ambitionbox.com/list-of-companies?page={PAGE}"
    driver.get(URL)

# wait for company cards
    next_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "a.pagination__next"))
    )
    next_button.click()

    # then wait for company cards to appear
    WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.companyCardWrapper"))
    )
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    for card in soup.select("div.companyCardWrapper"):
    # Company short name
        name_tag = card.select_one("h2.companyCardWrapper__companyName")
        name = name_tag.get_text(strip=True) if name_tag else None
        
        # Full name
        full_name = card.find("meta", {"itemprop": "alternateName"})
        full_name = full_name["content"] if full_name else None
        
        # Rating
        rating_tag = card.select_one("div.rating_text")
        rating = rating_tag.get_text(strip=True) if rating_tag else None
        
        # Reviews (ActionWrapper with title 'Reviews')
        reviews_tag = card.find("span", string="Reviews")
        reviews = reviews_tag.find_previous("span").get_text(strip=True) if reviews_tag else None
        
        # Jobs
        jobs_tag = card.find("span", string="Jobs")
        jobs = jobs_tag.find_previous("span").get_text(strip=True) if jobs_tag else None
        
        data.append({
            "Company": name,
            "Full Name": full_name,
            "Rating": rating,
            "Reviews": reviews,
            "Jobs": jobs
        })

df = pd.DataFrame(data)



driver.quit()


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0xc90c13+66051]
	GetHandleVerifier [0x0xc90c54+66116]
	(No symbol) [0x0xa6db33]
	(No symbol) [0x0xab78a8]
	(No symbol) [0x0xab7c4b]
	(No symbol) [0x0xafe0a2]
	(No symbol) [0x0xad9fd4]
	(No symbol) [0x0xafb7f1]
	(No symbol) [0x0xad9d86]
	(No symbol) [0x0xaab53e]
	(No symbol) [0x0xaac414]
	GetHandleVerifier [0x0xed8a13+2457603]
	GetHandleVerifier [0x0xed39d2+2437058]
	GetHandleVerifier [0x0xcb97f2+232930]
	GetHandleVerifier [0x0xca9a18+167944]
	GetHandleVerifier [0x0xcb092d+196381]
	GetHandleVerifier [0x0xc98ee8+99544]
	GetHandleVerifier [0x0xc99082+99954]
	GetHandleVerifier [0x0xc8322a+10266]
	BaseThreadInitThunk [0x0x74e25d49+25]
	RtlInitializeExceptionChain [0x0x773cd2fb+107]
	RtlGetAppContainerNamedObjectPath [0x0x773cd281+561]


In [14]:
df.shape

(20, 5)